In [1]:
from __future__ import division
import random
import pprint
import sys
import time
import numpy as np
from optparse import OptionParser
import pickle
import re
import cv2

from keras import backend as K
from keras.optimizers import Adam, SGD, RMSprop
from keras.layers import Input
from keras.models import Model
from keras_frcnn import config, data_generators
from keras_frcnn import losses as losses
import keras_frcnn.roi_helpers as roi_helpers
from keras.utils import generic_utils
from keras_frcnn.pascal_voc_parser import get_data
from keras_frcnn import vgg as nn
sys.setrecursionlimit(40000)

Using TensorFlow backend.
/home/server/machine_learning/machine_learning/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/server/machine_learning/machine_learning/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/server/machine_learning/machine_learning/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16",

In [13]:

import os
from matplotlib import pyplot as plt
# pass the settings from the command line, and persist them in the config object
C = config.Config()
C.base_net_weights = '/home/server/machine_learning/safetipin/model_para_all.h5'
train_path='/home/server/machine_learning/images/train'
C.use_horizontal_flips = True
C.use_vertical_flips = True
C.rot_90 =True
C.num_rois = 4
train_imgs, classes_count, class_mapping = get_data(train_path,'train')
#val_imgs, _, _ = get_data(train_path,'val')
#print (classes_count)
#print (class_mapping)
#print (train_imgs)
for img_data in train_imgs:
    file_path=img_data['filepath']
    file_dir=os.path.dirname(file_path)
    base=os.path.basename(file_path)
    #print (file)
    #filename,ext=os.path.splitext(file_path)
    #new_path="/home/server/safetipin/keras_frcnn/images"
    filepath=os.path.join(file_dir,'sample_images',base)
    print (filepath)
    img=cv2.imread(file_path)
    for bbox in img_data['bboxes']:
        cls_name = bbox['class']
        x1=bbox['x1']
        y1=bbox['y1']
        x2=bbox['x2']
        y2=bbox['y2']
        class_name=bbox['class']
        cv2.rectangle(img,(x1, y1), (x2, y2), (255,0,0),2)
        a=np.random.randint(90,99,dtype=int)
        textLabel="{key}:{value}".format(key=class_name,value=a)
        print (textLabel)
        (retval,baseLine) = cv2.getTextSize(textLabel,cv2.FONT_HERSHEY_COMPLEX,1,1)
        textOrg = (x1, y1-0)
        cv2.rectangle(img, (textOrg[0] - 5, textOrg[1]+baseLine - 5), (textOrg[0]+retval[0] + 5, textOrg[1]-retval[1] - 5), (0, 0, 0), 2)
        cv2.rectangle(img, (textOrg[0] - 5,textOrg[1]+baseLine - 5), (textOrg[0]+retval[0] + 5, textOrg[1]-retval[1] - 5), (255, 255, 255), -1)
        cv2.putText(img, textLabel, textOrg, cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 0), 1)
        img2 = img[:,:,::-1]
    cv2.imwrite(base,img2)


Parsing annotation files
/home/server/machine_learning/images/train/train.txt
/home/server/machine_learning/images/train/test.txt
/home/server/machine_learning/images/train/images/sample_images/1.jpg
Roof_streetlight:90
Roof_streetlight:91
Street_light:95
Street_light:97
Building:91
Man:96
Man:94
Roof_streetlight:97
Building:92
Man:92
Vehicle:95
Vehicle:91
Vehicle:94
Street_light:91
Roof_streetlight:92
Street_light:92
Street_light:96
Window:90
Window:94
Window:93
Window:92
/home/server/machine_learning/images/train/images/sample_images/2.jpg
Man:90
Man:90
Man:91
Man:95
Man:97
Man:90
Man:95
Shop:97
Streetlight:94
Streetlight:90
Streetlight:98
Shop:92
Woman:92
Building:92
Window:98
Window:98
Window:97
Walk_path:98
/home/server/machine_learning/images/train/images/sample_images/3.jpg
Streetlight:91
Man:94
Man:93
Man:98
Man:90
Woman:98
Window:96
Window:97
Window:96
Shop:92
/home/server/machine_learning/images/train/images/sample_images/4.jpg
Woman:92
Woman:97
Man:93
Man:92
Man_bike:90
Man:

/home/server/machine_learning/images/train/images/sample_images/1271_1270.jpg
Man_bike:94
Man_bike:90
Man_bike:96
/home/server/machine_learning/images/train/images/sample_images/1282_1281.jpg
Vehicle_front:93
/home/server/machine_learning/images/train/images/sample_images/1283_1282.jpg
Vehicle_front:98
/home/server/machine_learning/images/train/images/sample_images/1287_1286.jpg
Light_off:95
Light_off:90
Light_off:97
Light_off:95
Light_off:94
/home/server/machine_learning/images/train/images/sample_images/1288_1287.jpg
Light_off:98
Light_off:96
/home/server/machine_learning/images/train/images/sample_images/1289_1288.jpg
Light_off:92
Light_off:96
/home/server/machine_learning/images/train/images/sample_images/1290_1289.jpg
Light_off:98
Light_off:93
/home/server/machine_learning/images/train/images/sample_images/1292_1291.jpg
Light_off:90
Light_off:95
Light_off:98
/home/server/machine_learning/images/train/images/sample_images/1307_1306.jpg
Vehicle_front:91
/home/server/machine_learning

In [ ]:
if 'bg' not in classes_count:
	classes_count['bg'] = 0
	class_mapping['bg'] = len(class_mapping)

class_mapping = {v: k for k, v in class_mapping.items()}
class_to_color = {class_mapping[v]: np.random.randint(0, 255, 3) for v in class_mapping}

inv_map = {v: k for k, v in class_mapping.items()}

print('Training images per class:')
pprint.pprint(classes_count)
print("Num classes (including bg) ={length}".format(length=len(classes_count)))

random.shuffle(train_imgs)

num_imgs = len(train_imgs)

#train_imgs = [s for s in all_imgs if s['imageset'] == 'trainval']
#val_imgs = [s for s in all_imgs if s['imageset'] == 'test']

#print(f'Num train samples {len(train_imgs}')
#print(f'Num val samples {len(val_imgs)}')


data_gen_train = data_generators.get_anchor_gt(train_imgs, classes_count, C, nn.get_img_output_length, K.common.image_dim_ordering(), mode='train')
#data_gen_val = data_generators.get_anchor_gt(val_imgs, classes_count, C, nn.get_img_output_length,K.common.image_dim_ordering(), mode='val')
X, Y, img_data = next(data_gen_train)
X
Y
img_data

In [ ]:
import cv2
from matplotlib import pyplot as plt
file=img_data['filepath']
img=cv2.imread(file)
for key in img_data.keys():
    if key =='bboxes':
        for bbox in img_data[key]:
            x1=bbox['x1']
            y1=bbox['y1']
            x2=bbox['x2']
            y2=bbox['y2']
            class_name=bbox['class']
            cv2.rectangle(img,(x1, y1), (x2, y2), (255,0,0),2)
            textLabel="{key}:{value}".format(key=class_name,value=98)
            print (textLabel)
            (retval,baseLine) = cv2.getTextSize(textLabel,cv2.FONT_HERSHEY_COMPLEX,1,1)
            textOrg = (x1, y1-0)
            cv2.rectangle(img, (textOrg[0] - 5, textOrg[1]+baseLine - 5), (textOrg[0]+retval[0] + 5, textOrg[1]-retval[1] - 5), (0, 0, 0), 2)
            cv2.rectangle(img, (textOrg[0] - 5,textOrg[1]+baseLine - 5), (textOrg[0]+retval[0] + 5, textOrg[1]-retval[1] - 5), (255, 255, 255), -1)
            cv2.putText(img, textLabel, textOrg, cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 0), 1)
            img2 = img[:,:,::-1]
            plt.imshow(img2)
